Just a recap that when we were doing regression we had a column vector y so if we wanted the output to be of size (n_samples,1) and the weight vector is the same size then the X has to be of size (n_samples,n_features)
Exactly—spot on! Whenever you want one prediction per sample, you’ll aim for a result of shape `(n_samples,)` (or `(n_samples, 1)` if you keep it 2-D). In practice that means:

```text
   X.shape = (n_samples, n_features)
@  w.shape = (n_features, )    ⇒  y.shape = (n_samples, )
```

or equivalently

```text
   X.shape = (n_samples, n_features)
@  w_col.shape = (n_features, 1) ⇒  y.shape = (n_samples, 1)
```

Because matrix-multiply rules say:

- The “inner” dimensions must match (here: `n_features`).
- The “outer” dimensions define your output shape (here: `n_samples` × `1`).

So your mental checklist is:

1. **Count your samples** (rows of `X`).  
2. **Count your features** (cols of `X`).  
3. **Pick or reshape your weight vector** so it has `n_features` entries.  
4. **Multiply**—you’ll automatically get one result per sample.

---

### Quick sanity check in code

```python
import numpy as np

# 5 samples, 3 features
X = np.random.randn(5, 3)    # (5, 3)
w = np.array([0.2, -0.5, 1.0])  

y = X @ w                    # → shape (5,)
print(y.shape)               # (5,)
```

If you ever see something like `(3, 5)` or `(1, 5)` instead, you’ve flipped something—and a quick `.T` will fix it. Just remember: **outer dims give your output shape**!

---

# Neural Network as Curve-Fitter: A Matrix Summary

Below is a step-by-step breakdown of the tiny 1-input → 2-hidden → 1-output network you saw, showing how it really is “adding hills” to fit a curve. Jot this down in your notes!

---

## 1. Data & Shapes

- **Inputs**  
  - \(X\in\mathbb{R}^{n\times 1}\):  
    an \(n\)-by-1 column of dosages (one row per sample).

- **Hidden layer**  
  - Weights \(W\in\mathbb{R}^{1\times 2}\) (or equivalently \(\mathbb{R}^{2\times1}\) depending on convention):  
    two hidden neurons, one input each.  
  - Biases \(b\in\mathbb{R}^{\,2}\):  
    one bias per hidden neuron.  
  - Pre-activations  
    \[
      Z = X\,W \;+\; b
      \quad\Bigl(Z\in\mathbb{R}^{n\times2}\Bigr)
    \]
  - Activations (“hills”)  
    \[
      H = \sigma(Z)
      \quad\Bigl(H\in\mathbb{R}^{n\times2}\Bigr)
    \]
    where \(\sigma\) = ReLU, sigmoid, etc., applied element-wise.

- **Output layer**  
  - Weights \(v\in\mathbb{R}^{2\times1}\):  
    one weight for each hidden neuron feeding into the single output.  
  - Bias \(c\in\mathbb{R}\):  
    one bias for the lone output neuron.  
  - Final prediction  
    \[
      Y = H\,v \;+\; c
      \quad\Bigl(Y\in\mathbb{R}^{n\times1}\Bigr)
    \]
    giving one efficacy score per sample.

---

## 2. What’s Really Happening?

1. **Hidden layer**  
   - **Multiply** each input \(x\) by two slopes (\(w_1,w_2\)),  
   - **Shift** by biases \((b_1,b_2)\),  
   - **Squash** through \(\sigma\) → two “humps” \(h_1(x),h_2(x)\).

2. **Output layer**  
   - **Scale** those humps by \((v_1,v_2)\),  
   - **Sum** them,  
   - **Add** the single bias \(c\).  
   - Result: a composite curve that approximates your target!

---

## 3. Equations at a Glance

\[
\boxed{
\begin{aligned}
\text{Hidden pre-act:}~~& Z = X\,W + b, \\
\text{Hidden act:}~~~~~~& H = \sigma(Z), \\
\text{Output:}~~~~~~~~~~& Y = H\,v + c.
\end{aligned}
}
\]

- **Dimensions** must satisfy:
  - \(X: (n,1)\), \(W: (1,2)\) → \(Z: (n,2)\)  
  - \(H: (n,2)\), \(v: (2,1)\) → \(H\,v: (n,1)\)  
  - + bias \(c\) gives final \((n,1)\) predictions.

---

## 4. Role of Biases

- **Hidden biases \(b\)**: one per hidden neuron → vector \(\in\mathbb{R}^2\).  
- **Output bias \(c\)**: one for the single output neuron → scalar.  
- **Rule**: “# neurons in the layer” = “# bias terms in that layer.”

---

## 5. Why This Matters

- **Shape-checking**: Always confirm  
  ```python
  print(X.shape, W.shape, b.shape, v.shape)
  ```
  so your matrix multiplies line up and you get an \((n,1)\) output.
- **Universal Approximation**: More hidden neurons = more little hills → more flexible curves.
- **Readability**: Stick with “rows = samples, columns = features/neurons” and transpose only to fix mismatches.

---

🎉  **TL;DR**: A neural net = **sum of shifted+scaled activation “humps”** → matrix form is  
\[
Y = \sigma(XW + b)\,v + c.
\]  
Keep track of your shapes and biases, and you’ve got the full curve-fitting picture in one neat formula.

Sure—let’s write it out in the classic “layer-by-layer” notation you’ll see in most textbooks.

---

## 1-Hidden-Layer Network (1 → 2 → 1)

Let  
- \(x\in\mathbb{R}\) be your scalar input (dosage).  
- \(w_i, b_i\) be the weight and bias of hidden unit \(i\in\{1,2\}\).  
- \(v_i, c\) be the weight and bias of the single output neuron.

**Hidden layer pre-activation**  
\[
z_i^{[1]} \;=\; w_i\,x \;+\; b_i
\quad(i=1,2)
\]

**Hidden activation**  
\[
a_i^{[1]} \;=\; \sigma\bigl(z_i^{[1]}\bigr)
\quad\Longrightarrow\quad
a^{[1]} = 
\begin{bmatrix}a_1^{[1]} \\ a_2^{[1]}\end{bmatrix}
\]

**Output pre-activation**  
\[
z^{[2]} \;=\; \sum_{i=1}^{2} v_i\,a_i^{[1]} \;+\; c
\]

**Output activation (identity for regression)**  
\[
\hat y
=\; a^{[2]} \;=\; z^{[2]}
\]

Putting it all together:
\[
\boxed{ 
\hat y
= c \;+\; \sum_{i=1}^{2} v_i\,\sigma\bigl(w_i\,x + b_i\bigr)
}
\]

---

## Matrix Form

Define
\[
W^{[1]} = \begin{bmatrix}w_1\\ w_2\end{bmatrix},\quad
b^{[1]} = \begin{bmatrix}b_1\\ b_2\end{bmatrix},\quad
W^{[2]} = \begin{bmatrix}v_1 & v_2\end{bmatrix},\quad
b^{[2]} = \bigl[c\bigr].
\]
Then for an input vector \(x\) (or a batch \(X\in\mathbb R^{n\times1}\)),

1. **Layer 1**  
   \[
   z^{[1]} = W^{[1]}\,x \;+\; b^{[1]}, 
   \quad
   a^{[1]} = \sigma\!\bigl(z^{[1]}\bigr).
   \]
2. **Layer 2**  
   \[
   z^{[2]} = \bigl(W^{[2]}\,a^{[1]}\bigr) \;+\; b^{[2]},
   \quad
   a^{[2]} = z^{[2]}.
   \]

So compactly:
\[
\boxed{
a^{[2]} 
= W^{[2]}\,\sigma\!\bigl(W^{[1]}x + b^{[1]}\bigr)
\;+\; b^{[2]}.
}
\]

---

### General L-Layer Network

For layer \(\ell=1,\dots,L\), denote weights \(W^{[\ell]}\), biases \(b^{[\ell]}\), activations \(a^{[\ell]}\). Then

\[
\begin{aligned}
z^{[\ell]} &= W^{[\ell]}\,a^{[\ell-1]} + b^{[\ell]},\\
a^{[\ell]} &= 
\begin{cases}
\sigma\bigl(z^{[\ell]}\bigr), & \ell < L,\\[4pt]
\text{identity}(z^{[\ell]}), & \ell = L.
\end{cases}
\end{aligned}
\]
with \(a^{[0]} = x\).  

That’s the conventional equation set you’ll find in any deep-learning text—every network is just these two lines per layer!

Thought for a couple of seconds


Here’s the TL;DR in your own style:

* **Single sample** (3 inputs → 7 hidden → 2 output):

  ```
  (1×3) × (3×7) → (1×7)  → (1×7) × (7×2) → (1×2)
  ```

  * Multiply your 1×3 input by a 3×7 weight ⇒ 1×7
  * Add a 1×7 bias (broadcasting happens automatically) ⇒ still 1×7
  * Activate (ReLU, tanh, whatever) ⇒ 1×7
  * Multiply by a 7×2 weight ⇒ 1×2
  * Add a 1×2 bias ⇒ final 1×2 output

* **Batch of 5 samples** (3 inputs → 4 hidden → 2 output):

  ```
  X: (5×3)
  W₁: (3×4),  b₁: (1×4)    ⇒  Z₁ = X @ W₁ + b₁  → (5×4)
  W₂: (4×2),  b₂: (1×2)    ⇒  Z₂ = A₁ @ W₂ + b₂  → (5×2)
  ```

* **Bias shape** is always “1×number\_of\_nodes” (or a length-n\_nodes 1D array) so it neatly broadcasts across all your samples.

* **Alternative convention** (samples as columns):

  ```
  X: (3×5), W: (7×3), b: (7×1) ⇒ Z = W @ X + b → (7×5)
  ```

  but 99% of Python code uses rows-as-samples.

So yep—bias is always 1×\[#nodes], and your chain of matrix dims is exactly what you’ve been thinking. 🚀


In [2]:
import numpy as np

In [3]:
samples = np.array([
  [1, 2, 3],
  [6, 7, 8],
  [7, 8, 9],
  [3, 4, 5],
  [4, 5, 6],
])

# By convention, inputs in matrix form are often denoted by a capital X.
X = samples

targets = np.array([False, True, True, False, False])

In [27]:
uniques, indicies = np.unique(targets, return_inverse=True)
# Return inverse means that from the indecies returned in that array if you then use them to call the unique values, you can construct the original data.

In [ ]:
binary = uniques.shape[0]
n_samples = samples.shape[0]
y = np.zeros((n_samples,binary))

array([0, 1, 1, 0, 0], dtype=int64)

In [35]:
y[np.arange(n_samples),indicies] = 1
y

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [75]:
np.random.seed(10)

n_features = X.shape[1]
n_hidden = 4
w_h = np.random.uniform(-0.5,0.5,(n_features,n_hidden))
b_h = np.zeros((1,n_hidden))
b_h_show = np.ones((1,n_hidden))
print(np.dot(X,w_h))
print(X@w_h + b_h_show)

[[-0.72433282 -2.26463532  0.08585342  2.13004535]
 [-1.02964036 -8.09519327  0.17120801  6.94368505]
 [-1.09070187 -9.26130486  0.18827893  7.90641299]
 [-0.84645584 -4.5968585   0.11999526  4.05550123]
 [-0.90751735 -5.76297009  0.13706617  5.01822917]]
[[ 0.27566718 -1.26463532  1.08585342  3.13004535]
 [-0.02964036 -7.09519327  1.17120801  7.94368505]
 [-0.09070187 -8.26130486  1.18827893  8.90641299]
 [ 0.15354416 -3.5968585   1.11999526  5.05550123]
 [ 0.09248265 -4.76297009  1.13706617  6.01822917]]


In [76]:
h1 = X @ w_h + b_h
a1 = np.maximum(0,h1) # ReLu...
a1

array([[0.        , 0.        , 0.08585342, 2.13004535],
       [0.        , 0.        , 0.17120801, 6.94368505],
       [0.        , 0.        , 0.18827893, 7.90641299],
       [0.        , 0.        , 0.11999526, 4.05550123],
       [0.        , 0.        , 0.13706617, 5.01822917]])

In [77]:
np.random.seed(100)

w_o  = np.random.uniform(-0.5,0.5,(n_hidden,binary))
b_o = np.zeros((1,binary))
o = a1 @ w_o + b_o
o

array([[0.32118171, 0.66159156],
       [1.10083177, 2.19782851],
       [1.25676178, 2.5050759 ],
       [0.63304174, 1.27608634],
       [0.78897175, 1.58333373]])

In [ ]:
e_x = np.exp(o - o.max(axis=1, keepdims=True))

y_hat = e_x/np.sum(e_x,1,keepdims=True)
y_hat

array([[0.41570992, 0.58429008],
       [0.25030304, 0.74969696],
       [0.22299211, 0.77700789],
       [0.34455863, 0.65544137],
       [0.31123284, 0.68876716]])

array([[ 0.58429008, -0.58429008],
       [-0.25030304,  0.25030304],
       [-0.22299211,  0.22299211],
       [ 0.65544137, -0.65544137],
       [ 0.68876716, -0.68876716]])